<a href="https://colab.research.google.com/github/yuliiabosher/Fiber-optic-project/blob/statistical_analysis/DARKSPOTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#############
###########NEED TO RUN CODE TO LOAD POSTCODE LEVEL DATA ETC BEFORE THIS###############
# Find postcodes with < 1% coverage
# For each postcode, get their parliamentary consitiuency and group, then count no of postcodes
# Get total no postcodes for each constituency
#calculate what % have < 1% coverage and then display in a map
#############
columns_fast_broadband=['pcds', 'pcon',
       'Estimated Full Fibre availability (% premises)']

#get no of postcodes in each parliamentary constituency
df_numpostcodes = dfCombinedONSOFCOMData[columns_fast_broadband]
df_totalsnumpostcodes = df_numpostcodes.groupby(["pcon"], as_index=False)['pcds'].count()

#get the same data where postcode coverage < 1%
dfFastBroadbandData=dfCombinedONSOFCOMData[columns_fast_broadband]
dfFastBroadbandData.rename(columns={'Estimated Full Fibre availability (% premises)': 'FTTP'}, inplace=True)
dfVeryLowFibreData=dfFastBroadbandData.query('FTTP < 1')
dfGroupedVeryLowFibreData= dfVeryLowFibreData.groupby(["pcon"],as_index=False)['pcds'].count()

#merge the two dataframes
#df_europe_with_percent_geo_total = europe_cleaned.merge(df_final_total, left_on='NAME', right_on='Country', how='left')
df_final = dfGroupedVeryLowFibreData.merge(df_totalsnumpostcodes, left_on='pcon', right_on='pcon', how = 'left')
df_final['percentlow'] = df_final['pcds_x']/df_final['pcds_y']
df_final.sort_values(['percentlow'], ascending=False, inplace=True)




#now put it on a map

constituencies = gpd.read_file('https://github.com/yuliiabosher/Fiber-optic-project/raw/refs/heads/parliamentary-constituencies/Westminster_Parliamentary_Constituencies_Dec_2021_UK_BUC_2022_-8882165546947265805.zip')


constituencies_cleaned = constituencies[['PCON21CD', 'PCON21NM', 'geometry']]
constituencies_cleaned['PCON21NM'] = constituencies_cleaned['PCON21NM'].str.upper().str.strip()


fibre_by_constituency_geo_df = constituencies_cleaned.merge(df_final, left_on='PCON21CD', right_on='pcon')
fibre_by_constituency_geo_df.drop('PCON21CD', axis=1, inplace=True)

fig, ax = plt.subplots(1, 1)
fibre_by_constituency_geo_df.plot(column='percentlow', cmap='Blues', ax=ax, legend=True)
fig.set_size_inches(12,8)
plt.style.use('seaborn-v0_8-whitegrid')
font = {'family':'monospace','color':'black','size':10}
#ax.set_title('Areas with low fibre coverage', fontdict = font, pad=-75, loc='left')
plt.axis('off')
fig.text(x=0.23, y=0.77, s="Parliamentary constituencies", size=14)
fig.text(x=0.23, y=0.75, s="Proportion of postcodes < 1% fibre coverage", size=10)
plt.show()

#finall sort the dataframe and display the five constituencies with then highest proportion of postcodes with < 1%
fibre_by_constituency_geo_df.sort_values(['percentlow'], ascending=False, inplace=True)
df_table = fibre_by_constituency_geo_df
df_table['Proportion of postcodes < 1%'] = round(fibre_by_constituency_geo_df['percentlow']*100,2)
df_table['Parliamentary constituency'] = fibre_by_constituency_geo_df['PCON21NM']
df_table.drop('PCON21NM', axis=1, inplace=True)
df_table.drop('percentlow', axis=1, inplace=True)
df_table.drop('geometry', axis=1, inplace=True)
df_table.drop('pcon', axis=1, inplace=True)
df_table.drop('pcds_x', axis=1, inplace=True)
df_table.drop('pcds_y', axis=1, inplace=True)
display(df_table.head(5))
